# data to collect
got streaming data downloaded. good info
need to estimate game budget. probably with developer team size * time(month) * 10000
source: https://www.amazon.com/Blood-Sweat-Pixels-Triumphant-Turbulent-ebook/dp/B01NAKSWW1  
scrape:
review scores

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
import pprint
pp = pprint.PrettyPrinter()
from pymongo import MongoClient
from datetime import datetime

#import modules for web scraping
from __future__ import print_function, division
import requests
from bs4 import BeautifulSoup
import pickle

In [13]:
#load data
columns = ['current_views', 'stream_created_time', 'game_name', 'broadcaster_ID', 'broadcaster_name', 'delay_setting', 'follower_number', 'partner_status', 'broadcaster_language', 'total_views', 'language', 'broadcaster_created_time', 'playback_bitrate', 'source_resolution', 'stream_ID']
# for some reason it won't work without the index 'stream_ID' in the column list
# columns = columns_0[1:]
filename = 'Twitch_data' + '/all-2015-02-01-00-00-00.txt'
df = pd.read_csv(filename, sep = '\t', names = columns)
# dropping NA 'stream_ID' column
df = df.drop(columns = ['stream_ID'])
df.head(2)

,current_views,stream_created_time,game_name,broadcaster_ID,broadcaster_name,delay_setting,follower_number,partner_status,broadcaster_language,total_views,language,broadcaster_created_time,playback_bitrate,source_resolution
12932973168,34846,2015-02-01T02:48:25Z,Dota 2,29578325,beyondthesummit,-1,197236,True,en,197742366,en,2012-04-07T04:16:39Z,3247460,1600x900
12932549648,27293,2015-02-01T02:18:13Z,StarCraft II: Heart of the Swarm,30220059,esltv_sc2,-1,57446,True,en,49119702,en,2012-05-02T09:59:20Z,3172375,1920x1080


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12497 entries, 12932973168 to 12821263376
Data columns (total 14 columns):
current_views               12497 non-null int64
stream_created_time         12497 non-null object
game_name                   12496 non-null object
broadcaster_ID              12497 non-null int64
broadcaster_name            12497 non-null object
delay_setting               12497 non-null int64
follower_number             12497 non-null int64
partner_status              12497 non-null object
broadcaster_language        12497 non-null object
total_views                 12497 non-null int64
language                    12497 non-null object
broadcaster_created_time    12497 non-null object
playback_bitrate            12497 non-null int64
source_resolution           12497 non-null object
dtypes: int64(6), object(8)
memory usage: 1.4+ MB


In [15]:
df.describe()
#wow nice dense matrix

,current_views,broadcaster_ID,delay_setting,follower_number,total_views,playback_bitrate
count,12497.000000,1.249700e+04,12497.000000,12497.000000,1.249700e+04,1.249700e+04
mean,36.953749,5.598837e+07,-0.888933,1491.847323,1.988539e+05,1.616170e+06
std,626.634179,1.938797e+07,5.823741,17425.689377,3.897521e+06,1.013808e+06
min,0.000000,1.251900e+04,-1.000000,-1.000000,-1.000000e+00,-1.000000e+00
25%,0.000000,4.181871e+07,-1.000000,2.000000,3.600000e+01,9.429140e+05
50%,1.000000,5.808455e+07,-1.000000,13.000000,3.080000e+02,1.470023e+06
75%,4.000000,7.397475e+07,-1.000000,111.000000,2.408000e+03,2.281976e+06
max,34846.000000,8.150591e+07,540.000000,741293.000000,2.375675e+08,1.929220e+07


In [16]:
df.sort_values(by = 'total_views', ascending = False).head(10)[['broadcaster_name', 'game_name', 'total_views', 'language']]

,broadcaster_name,game_name,total_views,language
12932994272,starladder1,Dota 2,237567455,en
12932973168,beyondthesummit,Dota 2,197742366,en
12934563536,tsm_theoddone,XCOM: Enemy Within,171323063,en
12935159760,nl_kripp,Hearthstone: Heroes of Warcraft,102532780,en
12932696784,taketv,StarCraft II: Heart of the Swarm,83006063,de
12931574736,trick2g,League of Legends,79838633,en
12932512416,phantoml0rd,Counter-Strike: Global Offensive,74547798,en
12929261344,versuta,Dota 2,69994374,ru
12936030864,voyboy,League of Legends,65616654,en
12932549648,esltv_sc2,StarCraft II: Heart of the Swarm,49119702,en


In [ ]:
d = df.to_dict('index')
for key in d:
    d[key]['stream_ID'] = key
    print(d[key])
    # insert into Mongo
print(d)

In [6]:
# only concerned with top 10 games, gathered from annual twitch perspective, https://www.twitch.tv/year/2015/
game_list = [
'League of Legends', 
'Counter-Strike', 
'Dota 2',
'Hearthstone',
'Minecraft',
'H1Z1',
'Destiny',
'World of Tanks',
'World of Warcraft',
'FIFA 15'
]

In [2]:
df_list = []

In [3]:
client = MongoClient('localhost', 27017)

In [ ]:
client.list_database_names()

In [6]:
twitch_db = client['twitch_database']
twitch_collection = twitch_db.all_data

In [7]:
client.drop_database('twitter_database')

In [ ]:
columns = ['current_views', 'stream_created_time', 'game_name', 'broadcaster_ID', 'broadcaster_name', 'delay_setting', 'follower_number', 'partner_status', 'broadcaster_language', 'total_views', 'language', 'broadcaster_created_time', 'playback_bitrate', 'source_resolution', 'stream_ID']
# let's see if this works first
# put everything into mongo
file_list = os.listdir(path='Twitch_data')[1:]
df_list = []
master_df = pd.DataFrame()
for filename in tqdm(file_list[1000:]):
    time = filename
    filename = 'Twitch_data/' + filename
    try:
        df = pd.read_csv(filename, sep = '\t', names = columns)
        df = df.drop(columns = ['stream_ID'])
        d = df.to_dict('index')
        f = '%Y-%m-%d-%H-%M-%S'
        time = datetime.strptime(time[4:-4], f)
        for key in d:
            d[key]['stream_ID'] = key
            d[key]['time'] = time
            # insert into Mongo
            twitch_collection.insert_one(d[key])
    except:
        print(filename)
        

  0%|          | 6/9228 [03:18<89:14:29, 34.84s/it] 

Twitch_data/all-2015-02-04-08-15-00.txt


  0%|          | 44/9228 [18:34<64:35:01, 25.32s/it] 

In [103]:
# checking out just beyondthesummit
sample_streamer_df = master_df[master_df.broadcaster_ID == 29578325]
sample_streamer_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 753 entries, 12932973168 to 12983679024
Data columns (total 14 columns):
current_views               753 non-null object
stream_created_time         753 non-null object
game_name                   753 non-null object
broadcaster_ID              753 non-null object
broadcaster_name            753 non-null object
delay_setting               753 non-null object
follower_number             753 non-null object
partner_status              753 non-null object
broadcaster_language        753 non-null object
total_views                 753 non-null object
language                    753 non-null object
broadcaster_created_time    753 non-null object
playback_bitrate            753 non-null object
source_resolution           753 non-null object
dtypes: object(14)
memory usage: 88.2+ KB


In [106]:
sample_stream_viewers = sample_streamer_df[sample_streamer_df.stream_created_time == '2015-02-01T02:48:25Z']['current_views']
plt.plot()

12932973168    34846
12932973168    41662
12932973168    28948
12932973168    29816
12932973168    24613
12932973168    26989
12932973168    24907
12932973168    26243
12932973168    28627
12932973168    31900
12932973168    36021
12932973168    38553
12932973168    41141
12932973168    43448
12932973168    45137
12932973168    46591
12932973168    35436
12932973168    30498
12932973168    28087
12932973168    29713
12932973168    30787
12932973168    29899
12932973168    27100
12932973168    22306
12932973168    25090
12932973168    25031
12932973168    26640
12932973168    26633
12932973168    23619
12932973168    29043
               ...  
12932973168    58966
12932973168    60404
12932973168    62471
12932973168    64098
12932973168    65103
12932973168    66025
12932973168    68022
12932973168    69226
12932973168    45028
12932973168    39993
12932973168    35572
12932973168    36444
12932973168    37613
12932973168    39129
12932973168    33878
12932973168    42699
12932973168  

In [102]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12610739 entries, 12932973168 to 12749764544
Data columns (total 14 columns):
current_views               object
stream_created_time         object
game_name                   object
broadcaster_ID              object
broadcaster_name            object
delay_setting               object
follower_number             object
partner_status              object
broadcaster_language        object
total_views                 object
language                    object
broadcaster_created_time    object
playback_bitrate            object
source_resolution           object
dtypes: object(14)
memory usage: 1.4+ GB


In [83]:
import requests
import json

In [101]:
headers = {
    'Client-ID': 'c4gzslb0s9s2yevfvq5xtfr0y4k1gs',
}

params = (
    ('id', '29578325'),
    
)

url_user = 'https://api.twitch.tv/helix/users'
url_game = 'https://api.twitch.tv/helix/games'

response = requests.get(url_user, headers=headers, params=params)
print(response.status_code)
print(response.text)

200
{"data":[{"id":"29578325","login":"beyondthesummit","display_name":"BeyondTheSummit","type":"","broadcaster_type":"partner","description":"Beyond The Summit is a broadcasting studio and tournament organizer for a variety of esports titles (including Dota 2, SSBM, CS:GO, \u0026 DBFZ). Following and subscribing enables us to continue to create unique content and host amazing events like The Summit. Thank you for your support!","profile_image_url":"https://static-cdn.jtvnw.net/jtv_user_pictures/f170344d-7a60-47fa-8be6-a74e803fcf12-profile_image-300x300.jpg","offline_image_url":"https://static-cdn.jtvnw.net/jtv_user_pictures/f2633bad60b61448-channel_offline_image-1920x1080.jpeg","view_count":374545384}]}


In [21]:
#could try dask with this huge dataset, 5gb after zip or just somehow select part of the data
# Will put the files into Mongo in AWS instead, worry about dask or spark later
# import dask.dataframe as dd

In [3]:
# starting to scrape data here, try to write code here now can deal with using aws to scrape later
# test scrape from twitch
test_url = 'https://www.twitch.tv/c9sneaky'
response = requests.get(test_url)

In [4]:
response.status_code

200

In [6]:
page = response.text

In [7]:
soup = BeautifulSoup(page,"lxml")

In [8]:
print(soup.prettify())
# doesn't load the whole script

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <title>
   Twitch
  </title>
  <meta content="Twitch" property="og:site_name"/>
  <meta content="161273083968709" property="fb:app_id"/>
  <meta content="@twitch" property="twitter:site"/>
  <meta content="Twitch" property="og:title"/>
  <meta content="Twitch is the world's leading video platform and community for gamers." property="og:description"/>
  <meta content="//www-cdn.jtvnw.net/images/twitch_logo3.jpg" property="og:image"/>
  <meta content="https://www.twitch.tv/c9sneaky" property="og:url"/>
  <meta content="website" property="og:type"/>
  <link href="https://static.twitchcdn.net/assets/favicon-75270f9df2b07174c23c.ico" rel="icon"/>
  <link href="https://api.twitch.tv/" rel="dns-prefetch"/>
  <link href="https://gql.twitch.tv/" rel="dns-prefetch"/>
  <link href="https://cvp.twitch.tv/" rel="dns-prefetch"/>
  <link href="https://irc-ws.chat.twitch.tv/" rel="dns-prefetch"/>
  <link href="https://passport.twitch.tv/" rel=

In [57]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
chromedriver = '/Users/dynogravelso/Documents/GitHub/kojak_twitch_prediction/chromedriver' # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [87]:
# testpage sneaky
url = 'https://www.twitch.tv/c9sneaky'
driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [91]:
# scroll to the bottom to see if the stuff loads
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
#cleaning

In [ ]:
# eda

In [ ]:
#